In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb 
from xgboost import XGBClassifier, plot_importance


df = pd.read_csv('../input/train.csv')
#df.drop(columns=['IDH1', 'TP53'], inplace=True)

data = df.drop(columns=['ID', 'SUBCLASS'])
disease_list = df.SUBCLASS.unique()

df_test = pd.read_csv('../input/test.csv')
#df_test.drop(columns=['IDH1', 'TP53'], inplace=True)

test_ids = df_test['ID']
df_test.drop(columns=['ID'], inplace=True)

total = pd.concat([data, df_test])

In [2]:
total

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,R895R,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2542,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2543,WT,WT,WT,WT,WT,L217I,P221P P251P,R5M,G606D,I248Nfs,...,S2049Vfs S1909Vfs,L232R,WT,WT,L305L,WT,N252I N251I,G679V,WT,WT
2544,WT,WT,WT,WT,WT,WT,WT,WT,R1517H,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [3]:
def process_cell(value, col_name):
    if value == 'WT':  # 값이 'WT'이면 빈 문자열
        return ''
    elif isinstance(value, str):  # 문자열일 때는 첫 글자 + 컬럼명
        return  value[0]+ col_name
    elif isinstance(value, (int, float)):  # 숫자일 때는 첫 숫자(소수점 전) + 컬럼명
        return  str(value)[0]+col_name
    return value  # 그 외에는 원래 값 반환

# 각 셀을 처리하는 함수 적용
for col in total.columns:
    total[col] = total[col].apply(lambda x: process_cell(x, col))


In [4]:
total

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,RA2M,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,,,,,LABCA1,PABCA2,RABCA3,GABCA4,IABCA5,...,SZNF292,LZNF365,,,LZNFX1,,NZPBP,GZW10,,
2544,,,,,,,,,RABCA4,,...,,,,,,,,,,


In [5]:
nRows = df.shape[0]
data = total[:nRows]
df_test = total[nRows:]

# 데이터 변환 1차완료

In [ ]:
count_df = pd.DataFrame()
for d in disease_list:
    print(d)
    print(df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum).sort_values(ascending=False)[2:10])
    

In [99]:
df_test.apply(lambda x : x !='').agg(sum).sort_values(ascending=False)[:20]

TP53       1051
AHNAK       774
TCHH        734
APC         706
KMT2D       492
CACNA1A     482
SYNE1       444
MAGEE1      392
CPEB2       372
RYR2        361
KDM6B       356
TUBGCP6     354
PCLO        351
PIK3CA      343
RYR1        326
HLA-A       322
UBC         302
PLEC        284
CACNA1B     274
DST         262
dtype: int64

In [107]:
df[df['TCHH']!='WT']['SUBCLASS'].value_counts()

SUBCLASS
STES      41
SKCM      26
GBMLGG    19
UCEC      17
BRCA      16
COAD      15
BLCA      12
LUAD      12
CESC       9
LUSC       7
KIPAN      5
KIRC       5
HNSC       5
LIHC       5
TGCT       4
SARC       3
LGG        3
ACC        2
DLBC       2
PAAD       2
OV         2
PCPG       1
Name: count, dtype: int64

In [87]:
df[df.SUBCLASS=='GBMLGG'].apply(lambda x : x!='WT').agg(sum).sort_values(ascending=False)[:20]

SUBCLASS    461
ID          461
IDH1        189
TP53        169
ATRX         99
PTEN         75
EGFR         67
NF1          36
PCLO         34
RYR2         33
PIK3CA       31
PKHD1        26
SYNE1        23
SPTA1        23
TCHH         19
COL6A3       19
RELN         18
RB1          18
MYH2         16
NOTCH1       16
dtype: int64

In [3]:
df.iloc[577]['SUBCLASS']

'SKCM'

In [5]:
expert = pd.read_csv('../reference_data/BRCA.csv')
asso =pd.read_csv('../expert_r/BRCA-associ.csv')

upg = pd.read_csv('../expert_r/BRCA-upregulated.csv')
tup = set(upg['g.name'])
downg = pd.read_csv('../expert_r/BRCA-down.csv')
tdown = set(downg['g.name'])


In [46]:
 df[df['SUBCLASS']=='BRCA']

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
6,TRAIN_0006,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
18,TRAIN_0018,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
26,TRAIN_0026,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
28,TRAIN_0028,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
39,TRAIN_0039,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6157,TRAIN_6157,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6162,TRAIN_6162,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6165,TRAIN_6165,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6179,TRAIN_6179,BRCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [56]:

#for d in disease_list : 

tdf = df[df['SUBCLASS']=='BRCA']
td= tdf.apply(lambda x : x!='WT').agg(sum).reset_index(name='count').sort_values(by='count', ascending=False)
#print(len(td[td['count']>0]), tdf.shape[0])

t1 = set(td[td['count']>0][2:]['index'])

#td[td['count']>5]
td[td['count']>0][1:]#['index']


,index,count
1,SUBCLASS,786
2979,PIK3CA,250
4109,TP53,218
702,CDH1,73
1553,GATA3,58
...,...,...
2604,NDUFB3,1
2594,NDUFA4,1
1217,ECI1,1
1218,ECI2,1


In [71]:
df_test['COL12A1'].value_counts()

COL12A1
                 2383
R392H               2
G3058S G1894S       2
R1924H R760H        2
R1087C R2251C       2
                 ... 
P1588H V1266M       1
G1304G G140G        1
G13G                1
V2480M V1316M       1
T1046Hfs            1
Name: count, Length: 157, dtype: int64

In [65]:
df_test[df_test['S100A12']!='']

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,,,,,,,,,,,...,,,,,,,,,,
85,,,,,,,,,,,...,,,,,,,,,,
312,M1V M101V,,,,,T823A,,,D493N,E1411X,...,F1944C F1804C,,,,,,R247Q R246Q,,,
956,,,,,,L2099I,,,,E250X,...,R2151C R2291C,,,,,,,K484N,,
1316,,,,,,,,,,,...,,,,,,,,,,
1598,,,,,,P1478L,S1021S,,P47S,L343F,...,,,,,,,,,,
1999,,,,,,,M1996T M2026T,,S1696R,,...,R2405Gfs R2545Gfs,,,,,,,L256L,,
2446,R1031Q,,,,,R1897R,,,P381S,,...,,,,,,,,,,
2467,,,,,,,,,,,...,,,,,,,,,,


In [77]:
df['COL12A1'].value_counts().reset_index().sort_values(by='COL12A1', ascending=False).to_csv('col12a1.csv')

In [67]:
df[df['S100A12']!='WT']


,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1369,TRAIN_1369,SKCM,E1311K E342K R147C E1311K E342K R147C,P307S T163T P307S T163T,P244L P244L,WT,P239S P239S,P91S P91S,S1492F A1262A F1948F S1237F L1683L S1492F A126...,G495G P72L L310L D1089D G495G P72L L310L D1089D,...,P873S L2423V R2173* S1473F P873S L2423V R2173*...,E54K E54K,WT,WT,F380F S197F F380F S197F,G378S L264L P349P G121I I207I G378S L264L P349...,E143K E143K,L489L L489L,WT,WT
1747,TRAIN_1747,STES,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1908,TRAIN_1908,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,S2580F,WT,WT,WT,E1124E,WT,WT,WT,WT,WT
2189,TRAIN_2189,UCEC,F1389F T1239T V129V L13I,WT,S32L,WT,WT,R1901I Q902R,WT,S947S R661Q P440P,...,D193N D1372Y,WT,WT,WT,P802L R310*,WT,R258I V60V,WT,WT,WT
2909,TRAIN_2909,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4283,TRAIN_4283,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4543,TRAIN_4543,SKCM,WT,WT,WT,WT,WT,P1478S,WT,W704*,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4588,TRAIN_4588,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4737,TRAIN_4737,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
5049,TRAIN_5049,UCEC,K271N N159Y,WT,WT,WT,R436*,I1662I D1411Y,F1897F E1289K,R998C F391F R43R,...,R89Q L1065I V2150V R2335Q,R167Q,WT,R59C,L465I,WT,WT,S161Y,WT,WT


0,"['ABCG4', 'ADAM2', 'AHNAK', 'ALMS1', 'C5', 'CD44', 'CDK7', 'CDKAL1', 'CEP131', 'CLDN16', 'COL12A1', 'CYFIP2', 'DCC', 'DLC1', 'EIF2S1', 'FMO3', 'GSR', 'HAS2', 'HCCS', 'HDAC9', 'HDLBP', 'HLA-DRB1', 'IDH3B', 'KDM4B', 'KMT2D', 'LLGL2', 'MAGEE1', 'MFHAS1', 'MLYCD', 'MNAT1', 'MYC', 'MYH1', 'MYL1', 'MYOM2', 'OAT', 'PABPC1', 'PBX2', 'PDK4', 'PUS1', 'RELN', 'S100A12', 'SEC31A', 'SHH', 'SIRPA', 'SOCS3', 'SYNE1', 'TCHH', 'TFPI2', 'TIMP3', 'TP53', 'TUBA3C', 'UBC']"


In [69]:
df[df['COL12A1']!='WT']

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
24,TRAIN_0024,COAD,WT,WT,WT,WT,WT,WT,WT,WT,...,G528*,WT,WT,WT,T1904M,WT,WT,WT,WT,WT
47,TRAIN_0047,SKCM,E1462K,WT,E66K,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
71,TRAIN_0071,UCEC,K289T,WT,WT,WT,WT,S824L R130I,WT,D551Y,...,E1936* K2344N,WT,WT,WT,F1111L,WT,WT,E529G,WT,WT
74,TRAIN_0074,STES,D533D,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,A56V,WT,WT
76,TRAIN_0076,LUSC,WT,WT,WT,WT,WT,K1522N,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6072,TRAIN_6072,STES,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6131,TRAIN_6131,DLBC,WT,WT,WT,WT,WT,WT,WT,R1474W,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
6153,TRAIN_6153,SKCM,D636N,WT,WT,WT,WT,WT,WT,D342N E506K P1301S,...,WT,WT,WT,WT,WT,E317K,WT,WT,WT,WT
6156,TRAIN_6156,COAD,S150S,WT,WT,WT,WT,D124G,WT,L405I,...,WT,WT,WT,WT,R409Q,WT,WT,WT,D186Y,WT


In [ ]:
t1.intersection(set(asso['g.name']))

In [40]:
#하위개수 1500
len(set(td[td['count']>0][1500:]['index']).intersection(tup))

25

In [41]:
len(set(td[td['count']>0][1500:]['index']).intersection(tdown))

28

In [43]:
len(set(td[td['count']>0][1500:]['index']).intersection(set(asso['g.name'])))

104

In [45]:
len(set(td[td['count']>0][1500:]['index']).intersection(set(expert['g.name'])))

152

In [47]:
#상위 개수
set(td[td['count']>0][2:1500]['index']).intersection(tup)

{'CCNB2',
 'CDH11',
 'CDKN3',
 'CENPF',
 'CIT',
 'CKAP5',
 'COL11A1',
 'COL12A1',
 'COL5A1',
 'COPB2',
 'CSTF3',
 'CTSB',
 'E2F3',
 'E2F8',
 'ECT2',
 'ENTPD7',
 'EPSTI1',
 'FN1',
 'LSR',
 'MAPK13',
 'MELK',
 'MKI67',
 'MUC1',
 'PARP12',
 'PRC1',
 'RACGAP1',
 'RAD54L',
 'RNF213',
 'S100A11',
 'SDC1',
 'SDF2L1',
 'SLC1A4',
 'SLC20A1',
 'SMC4',
 'SQLE',
 'STAT1',
 'TIMELESS',
 'TOP2A',
 'TTK'}

In [55]:
td[td['count']>0][:1500]['index']

0             ID
1       SUBCLASS
4109        TP53
726       CDKN2A
3470        RYR1
          ...   
680         CD99
1556        GBP2
1557        GBP4
1561        GCDH
1548      GAPDHS
Name: index, Length: 1500, dtype: object

In [48]:
set(td[td['count']>0][2:1500]['index']).intersection(tdown)

{'ABLIM1',
 'ACSL5',
 'ADAMTS1',
 'AHNAK',
 'ANXA1',
 'BDH2',
 'CNBP',
 'DMD',
 'DPYSL2',
 'DST',
 'DUSP6',
 'EFEMP1',
 'EGFR',
 'ETS2',
 'F10',
 'GPAM',
 'GPC3',
 'GPRC5B',
 'IRS2',
 'ITIH5',
 'KIF1B',
 'KLF4',
 'LAMA3',
 'LEPR',
 'LIFR',
 'MATN2',
 'PKD2',
 'PROS1',
 'SFRP1',
 'SGCE',
 'SOCS2',
 'SORBS1',
 'SPTBN1',
 'SRPX',
 'STXBP1',
 'TGFBR2',
 'TNS1',
 'ZBTB16'}

In [ ]:
set(td[td['count']>0][1:1500]['index']).intersection(set(asso['g.name']))

In [ ]:
set(td[td['count']>0][2:1500]['index']).intersection(set(expert['g.name']))

In [31]:

#for d in disease_list : 

tdf = df[df['SUBCLASS']=='PAAD']
td= tdf.apply(lambda x : x!='WT').agg(sum).reset_index(name='count').sort_values(by='count', ascending=False)
#print(len(td[td['count']>0]), tdf.shape[0])

t1 = set(td[td['count']>0][2:]['index'])

#td[td['count']>5]
td[td['count']>0][2:]#['index']


,index,count
4109,TP53,79
726,CDKN2A,19
3470,RYR1,12
1620,GNAS,9
3020,PLEC,9
...,...,...
2699,NOLC1,1
2700,NOP14,1
2701,NOP16,1
2702,NOP2,1


In [25]:
len(t1.intersection(tup))

85

In [26]:
len(t1.intersection(tdown))

86

In [28]:
len(t1.intersection(set(asso['g.name'])))

262

In [29]:
len(t1.intersection(set(expert['g.name'])))

407

In [65]:
df[df.SUBCLASS=='BRCA']['RNF213'].apply(lambda x : x!='WT').agg(sum)

26

{'AHNAK', 'DMD', 'DST'}

In [66]:
df[df.SUBCLASS=='BRCA']['AHNAK'].apply(lambda x : x!='WT').agg(sum)

24

In [67]:
df[df.SUBCLASS=='BRCA']['DMD'].apply(lambda x : x!='WT').agg(sum)

34

In [68]:
df[df.SUBCLASS=='BRCA']['DST'].apply(lambda x : x!='WT').agg(sum)

29

In [ ]:
t1.intersection(set(expert['g.name']))

In [62]:
t1.intersection(set(asso['g.name']))

{'CDH1', 'GATA3', 'MAP3K1', 'NOTCH2', 'PIK3CA', 'PTEN', 'SYNE1', 'TP53'}

In [ ]:

for g in asso['g.name']:
    try :
        print(df[df.SUBCLASS=='BRCA'][g].apply(lambda x : x!='WT').agg(sum))
    except KeyError:
        print(g)

In [3]:
data

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,R895R,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,,,,,,,,,,,...,,,,,,,,,,
6197,,,,,,,,,,,...,,,,,,,,,,
6198,,,,,,,,,,,...,,,,,,,,,T181S,
6199,,,,,,,,,,,...,,,,,,,,,,


In [4]:
df_test

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,R587Q,,,,,...,,,,,,I383Sfs,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,,,,,L217I,P221P P251P,R5M,G606D,I248Nfs,...,S2049Vfs S1909Vfs,L232R,,,L305L,,N252I N251I,G679V,,
2544,,,,,,,,,R1517H,,...,,,,,,,,,,


In [5]:
#1. train data에서 모두 WT인 거 삭제하기 
#td = data.apply(lambda x : x=='WT').agg(sum , axis=1).sort_values(ascending=False) == data.shape[1]
#df.drop(list(td[td].index), inplace=True)

#data = df.drop(columns=['ID', 'SUBCLASS'])


In [6]:
#df.iloc[td[td].index].SUBCLASS.value_counts()

In [7]:
#too many 변이, ROWs DELETE
#for d in disease_list:
#    td = df[df['SUBCLASS']==d].apply(lambda x : x!='WT').agg(sum , axis=1).sort_values(ascending=False) >= 800 #706
#    if td[td].sum()> 0:
#        print(d, list(td[td].index))
#        df.drop(list(td[td].index), inplace=True)


# Feature 선택을 위한 데이터 2차변환

In [6]:
y = df['SUBCLASS']
encoder2 = LabelEncoder()
y = encoder2.fit_transform(y)

In [7]:
total

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,RA2M,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,,,,,,,,,,,...,,,,,,,,,,
2542,,,,,,,,,,,...,,,,,,,,,,
2543,,,,,,LABCA1,PABCA2,RABCA3,GABCA4,IABCA5,...,SZNF292,LZNF365,,,LZNFX1,,NZPBP,GZW10,,
2544,,,,,,,,,RABCA4,,...,,,,,,,,,,


In [8]:
# 데이터프레임을 셀 단위로 고유값으로 변환
unique_values, unique_indices = pd.factorize(total.values.ravel())

# 고유값을 다시 데이터프레임 모양으로 변환
total_transformed = pd.DataFrame(unique_values.reshape(total.shape), columns=total.columns)

print(total_transformed)

      A2M  AAAS  AADAT  AARS1  ABAT  ABCA1  ABCA2  ABCA3  ABCA4  ABCA5  ...  \
0       0     0      0      0     0      0      0      0      0      0  ...   
1       0     0      0      0     0      0      0      0      0      0  ...   
2      43     0      0      0     0      0      0      0      0      0  ...   
3       0     0      0      0     0      0      0      0      0      0  ...   
4       0     0      0      0     0      0      0      0      0      0  ...   
...   ...   ...    ...    ...   ...    ...    ...    ...    ...    ...  ...   
8742    0     0      0      0     0      0      0      0      0      0  ...   
8743    0     0      0      0     0      0      0      0      0      0  ...   
8744    0     0      0      0     0   7626  11169   5939   5475   2269  ...   
8745    0     0      0      0     0      0      0      0   7967      0  ...   
8746    0     0      0      0     0      0      0      0      0      0  ...   

      ZNF292  ZNF365  ZNF639  ZNF707  ZNFX1  ZNRF4 

In [9]:
total_transformed

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,43,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8742,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8744,0,0,0,0,0,7626,11169,5939,5475,2269,...,7835,23232,0,0,17683,0,43886,8099,0,0
8745,0,0,0,0,0,0,0,0,7967,0,...,0,0,0,0,0,0,0,0,0,0


# Feature selection - method with XGB


In [11]:
nRows = df.shape[0]
X = total_transformed[:nRows]
df_test = total_transformed[nRows:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156)  

# 2000개 feature 선택

In [12]:
from sklearn.metrics import accuracy_score
# Step 5: Train an XGBoost model to find important features
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Get feature importances
feature_importances = xgb_model.feature_importances_
sorted_idx = feature_importances.argsort()

# Select top 2000 features based on importance
top_n = 2000  # Adjust this value to select the top N important features
top_features_idx = sorted_idx[-top_n:]

# Step 6: Reduce training and testing data to the top N features
X_train_reduced = X_train.iloc[:, top_features_idx]
X_test_reduced = X_test.iloc[:, top_features_idx]

# Step 7: Train a new XGBoost model on the reduced feature set
best_xgb = xgb.XGBClassifier(n_estimators=100, random_state=42)
best_xgb.fit(X_train_reduced, y_train)

# Step 8: Evaluate the model on the test set
y_pred = best_xgb.predict(X_test_reduced)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the reduced feature set: {accuracy}")

Accuracy on the reduced feature set: 0.323481998925309


In [13]:
top_features_idx

array([2405, 2408, 2409, ..., 1472, 3551, 3578])

In [15]:
total_transformed2 = total_transformed.iloc[:, top_features_idx]

## ====##  # 선택된 Feature, total_transformed2에 적용 완료


In [ ]:
X_test_final = df_test.iloc[:, top_features_idx]

In [ ]:
y_test_pred = best_xgb.predict(X_test_final)

In [ ]:
y_test_pred

array([21, 25,  2, ...,  3, 10, 11])

In [ ]:

# Step 11: Decode the predicted class labels back to their original subclass names
y_test_pred_labels = encoder2.inverse_transform(y_test_pred)
res_df0 = pd.DataFrame()
res_df0['ID'] = test_ids
# Step 12: Store the predictions in the test DataFrame
res_df0['SUBCLASS'] = y_test_pred_labels

# Display the first few rows of predictions
res_df0[['SUBCLASS']].value_counts()


SUBCLASS
BRCA        559
COAD        432
KIPAN       230
STES        188
GBMLGG      118
HNSC         96
OV           93
KIRC         87
UCEC         80
SKCM         73
LGG          70
THCA         66
CESC         59
PRAD         53
LIHC         50
TGCT         50
PCPG         47
PAAD         41
SARC         40
LAML         29
LUAD         26
LUSC         18
BLCA         18
ACC          16
THYM          7
Name: count, dtype: int64

In [ ]:
#res_df0[['ID', 'SUBCLASS']].to_csv('./XGB_FEATURESELECTION.csv',index=False)


# ====

# DMatrix, XGBClassifier 적용

In [16]:
nRows = df.shape[0]
X = total_transformed2[:nRows]
df_test = total_transformed2[nRows:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=156) #shuffle=False,stratify = y, 
print(X_train.shape, X_test.shape)

(4340, 2000) (1861, 2000)


In [17]:
dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)

In [18]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    'num_class': 26
}
num_boost_round = 50

dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)
model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_boost_round , early_stopping_rounds=5, 
                  evals=[(dtrain, 'train'), (dtest, 'valid')],
                  verbose_eval=2)

[0]	train-mlogloss:2.62561	valid-mlogloss:2.72368
[2]	train-mlogloss:2.23077	valid-mlogloss:2.45934
[4]	train-mlogloss:2.00035	valid-mlogloss:2.33901
[6]	train-mlogloss:1.84366	valid-mlogloss:2.26303


In [250]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    #'learning_rate': 0.05
}
num_boost_round = 1000

clf = xgb.XGBClassifier( n_estimators = 100,        # 부스팅 단계 수
                        max_depth = 6,             # 개별 트리의 최대 깊이
                        learning_rate = 0.1,       # 학습률, 낮출수록 느리게 학습하지만 성능 개선 가능
                        #subsample=0.8,           # 각 부스팅 단계에서 사용할 샘플의 비율
                        #scale_pos_weight = class_weight_dict,  # 클래스별 가중치
                        #colsample_bytree=0.8,    # 각 트리에서 사용할 특성의 비율
                        random_state = 42,         # 랜덤 시드
                        n_jobs = -1,             # 모든 CPU 코어 사용,
                        early_stopping_rounds = 5,
                        **params)
clf.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test,y_test)],
        verbose=2);

[0]	validation_0-mlogloss:3.02717	validation_1-mlogloss:3.06005
[2]	validation_0-mlogloss:2.74692	validation_1-mlogloss:2.83452
[4]	validation_0-mlogloss:2.56054	validation_1-mlogloss:2.69797
[6]	validation_0-mlogloss:2.41873	validation_1-mlogloss:2.59815
[8]	validation_0-mlogloss:2.30340	validation_1-mlogloss:2.52437
[10]	validation_0-mlogloss:2.20632	validation_1-mlogloss:2.46401
[12]	validation_0-mlogloss:2.12243	validation_1-mlogloss:2.41453
[14]	validation_0-mlogloss:2.05068	validation_1-mlogloss:2.37431
[16]	validation_0-mlogloss:1.98660	validation_1-mlogloss:2.34064
[18]	validation_0-mlogloss:1.92996	validation_1-mlogloss:2.31120
[20]	validation_0-mlogloss:1.87882	validation_1-mlogloss:2.28688
[22]	validation_0-mlogloss:1.83270	validation_1-mlogloss:2.26589
[24]	validation_0-mlogloss:1.79020	validation_1-mlogloss:2.24736
[26]	validation_0-mlogloss:1.75206	validation_1-mlogloss:2.23174
[28]	validation_0-mlogloss:1.71684	validation_1-mlogloss:2.21716
[30]	validation_0-mlogloss:1.6

In [256]:
y_true = y_test
y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)
y_true.shape, y_pred.shape, y_score.shape

((1861,), (1861,), (1861, 26))

In [ ]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

In [257]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.3449758194519076

In [ ]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

In [258]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

0.8542248415339827

In [263]:
df_test

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
6201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6202,0,0,0,0,0,10117,0,0,0,0,...,0,0,0,0,0,13934,0,0,0,0
6203,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6204,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8742,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8744,0,0,0,0,0,7626,11169,5939,5475,2269,...,7835,23232,0,0,17683,0,43886,8099,0,0
8745,0,0,0,0,0,0,0,0,7967,0,...,0,0,0,0,0,0,0,0,0,0


In [264]:
clf.predict(df_test)

array([21, 25,  2, ...,  3, 10, 11])

In [266]:
res = clf.predict(df_test)
res = encoder2.inverse_transform(res)

df_test1 = pd.read_csv('../input/test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df


,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,UCEC
2,TEST_0002,BRCA
3,TEST_0003,LGG
4,TEST_0004,STES
...,...,...
2541,TEST_2541,COAD
2542,TEST_2542,PAAD
2543,TEST_2543,CESC
2544,TEST_2544,LAML


In [267]:
res_df.SUBCLASS.value_counts()

SUBCLASS
BRCA      656
COAD      532
KIPAN     221
STES      156
OV        139
GBMLGG    117
UCEC       83
THCA       71
LGG        70
HNSC       69
SKCM       67
KIRC       63
TGCT       40
CESC       37
LIHC       35
LAML       30
PRAD       29
BLCA       21
LUSC       20
PAAD       19
LUAD       18
ACC        17
PCPG       14
SARC       13
THYM        5
DLBC        4
Name: count, dtype: int64

In [268]:
res_df.to_csv('v3.csv',index = False)

# 1차 결과 저장 완료

In [145]:
diseaseList = df.SUBCLASS.value_counts().rename_axis('disease').reset_index(name='counts')
diseaseList

,disease,counts
0,BRCA,786
1,KIPAN,515
2,GBMLGG,461
3,STES,379
4,KIRC,334
5,THCA,324
6,SKCM,276
7,PRAD,266
8,OV,253
9,LGG,229


In [159]:
X['SUBCLASS'] = df.SUBCLASS
X

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX,SUBCLASS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,KIPAN
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,SARC
2,182,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,SKCM
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,KIRC
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,GBMLGG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,LUAD
6197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,LGG
6198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,60,0,COAD
6199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,TGCT


KeyError: 'SUBCLASS'

In [7]:
y


array([ 8, 19, 20, ...,  4, 22, 20])

In [8]:
X['SUBCLASS'] = y

In [9]:
X

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX,SUBCLASS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19
2,182,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,13
6197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
6198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,60,0,4
6199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22


In [127]:
selected_genes = set()
for d in disease_list:
    tdf = df [ df['SUBCLASS'].isin([d] )]
    total_len = tdf.shape[0]
    
    tdf = tdf.apply(lambda x : x !='WT').agg(sum)
    tdf = tdf.iloc[2:].reset_index(name='count').sort_values(by='count', ascending=False)
    #thr[d]
    rlist =[]
    if total_len > 200: 
        rlist = tdf.loc[tdf['count'] > 5]['index'].to_list()
    else :
        rlist = tdf.loc[tdf['count'] > 5]['index'].to_list()

    print(d, len(rlist))
    selected_genes.update(rlist)
selected_genes = list(selected_genes)
len(selected_genes)

KIPAN 286
SARC 1547
SKCM 2112
KIRC 68
GBMLGG 147
STES 2034
BRCA 691
THCA 4
LIHC 2394
HNSC 334
PAAD 2741
OV 47
PRAD 16
UCEC 4003
LAML 316
COAD 936
ACC 1580
LGG 16
LUSC 3330
LUAD 3123
CESC 2905
PCPG 524
THYM 388
BLCA 2931
TGCT 1366
DLBC 1657


4218

In [59]:
def get_gene_list1():
    selected_genes = set()
    for d in disease_list:
        count_df = df[df.SUBCLASS==d].apply(lambda x : x!='WT').agg(sum)
        count_df = count_df[2:].reset_index(name='count')        
        gene_list = count_df[count_df['count']>5]['index'].values
        print(d, len(gene_list))
        selected_genes.update(gene_list)
    print(len(selected_genes))
    return selected_genes
glist = get_gene_list1()

KIPAN 286
SARC 31
SKCM 2112
KIRC 68
GBMLGG 147
STES 2034
BRCA 691
THCA 4
LIHC 106
HNSC 334
PAAD 22
OV 47
PRAD 16
UCEC 2006
LAML 9
COAD 936
ACC 51
LGG 16
LUSC 587
LUAD 468
CESC 175
PCPG 3
THYM 1
BLCA 209
TGCT 14
DLBC 17
2925


In [128]:
if 'SUBCLASS' in selected_genes:
    print('ry')

In [129]:

data = data.loc[:, list(selected_genes)]
df_test = df_test.loc[:, list(selected_genes)]


In [130]:
df_test

,PDLIM4,SLC6A12,ADD2,CAMKK2,HNRNPA2B1,LAMP2,TDRD7,FGF22,GMPR,CRTAM,...,DYRK2,DMC1,HSD17B6,NAP1L1,USH1C,NAMPT,SNTB1,PARVA,DLL1,BLVRB
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,37,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2543,0,0,0,58,0,0,2,0,0,0,...,32,31,20,0,0,16,0,29,0,0
2544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Multi-class classification

In [131]:
dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)

In [132]:
params = {
    'tree_method': 'approx',
    'objective':  'multi:softprob',
    'num_class': 26
}
num_boost_round = 50

dtrain = xgb.DMatrix(data = X_train, label= y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)
model = xgb.train(params=params, dtrain=dtrain, num_boost_round=num_boost_round , early_stopping_rounds=5, 
                  evals=[(dtrain, 'train'), (dtest, 'valid')],
                  verbose_eval=2)

[0]	train-mlogloss:2.60254	valid-mlogloss:2.70059
[2]	train-mlogloss:2.20517	valid-mlogloss:2.44158
[4]	train-mlogloss:1.97586	valid-mlogloss:2.32014
[6]	train-mlogloss:1.81681	valid-mlogloss:2.25111
[8]	train-mlogloss:1.69312	valid-mlogloss:2.20809
[10]	train-mlogloss:1.59590	valid-mlogloss:2.17595
[12]	train-mlogloss:1.51923	valid-mlogloss:2.15672
[14]	train-mlogloss:1.45402	valid-mlogloss:2.14424
[16]	train-mlogloss:1.39989	valid-mlogloss:2.13541
[18]	train-mlogloss:1.35358	valid-mlogloss:2.13200
[20]	train-mlogloss:1.31120	valid-mlogloss:2.12959
[22]	train-mlogloss:1.27352	valid-mlogloss:2.12840
[24]	train-mlogloss:1.23913	valid-mlogloss:2.12604
[26]	train-mlogloss:1.20742	valid-mlogloss:2.12326
[28]	train-mlogloss:1.17780	valid-mlogloss:2.12503
[30]	train-mlogloss:1.15042	valid-mlogloss:2.12658
[32]	train-mlogloss:1.12529	valid-mlogloss:2.12676


In [133]:
params = {
    'tree_method': 'approx',
    'objective': 'multi:softprob',
    #'learning_rate': 0.05
}

clf = xgb.XGBClassifier(n_estimators = 100,        # 부스팅 단계 수
                        max_depth = 8,             # 개별 트리의 최대 깊이
                        learning_rate = 0.1,       # 학습률, 낮출수록 느리게 학습하지만 성능 개선 가능
                        subsample=0.8,           # 각 부스팅 단계에서 사용할 샘플의 비율
                        #scale_pos_weight = class_weight_dict,  # 클래스별 가중치
                        colsample_bytree= 0.8,    # 각 트리에서 사용할 특성의 비율
                        random_state = 42,         # 랜덤 시드
                        n_jobs = -1,             # 모든 CPU 코어 사용,
                        early_stopping_rounds = 5,
                        **params)
clf.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test,y_test)],
        verbose=2);

[0]	validation_0-mlogloss:3.05657	validation_1-mlogloss:3.08553
[2]	validation_0-mlogloss:2.79722	validation_1-mlogloss:2.89140
[4]	validation_0-mlogloss:2.60318	validation_1-mlogloss:2.74851
[6]	validation_0-mlogloss:2.44986	validation_1-mlogloss:2.64181
[8]	validation_0-mlogloss:2.31623	validation_1-mlogloss:2.55582
[10]	validation_0-mlogloss:2.20721	validation_1-mlogloss:2.48938
[12]	validation_0-mlogloss:2.10960	validation_1-mlogloss:2.43218
[14]	validation_0-mlogloss:2.02645	validation_1-mlogloss:2.38866
[16]	validation_0-mlogloss:1.95356	validation_1-mlogloss:2.34867
[18]	validation_0-mlogloss:1.88601	validation_1-mlogloss:2.31507
[20]	validation_0-mlogloss:1.82574	validation_1-mlogloss:2.28466
[22]	validation_0-mlogloss:1.76914	validation_1-mlogloss:2.26092
[24]	validation_0-mlogloss:1.71807	validation_1-mlogloss:2.24103
[26]	validation_0-mlogloss:1.67156	validation_1-mlogloss:2.22383
[28]	validation_0-mlogloss:1.62690	validation_1-mlogloss:2.20910
[30]	validation_0-mlogloss:1.5

KeyboardInterrupt: 

In [64]:
y_true = y_test
y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)
y_true.shape, y_pred.shape, y_score.shape

((1833,), (1833,), (1833, 26))

In [65]:
from sklearn import metrics 
metrics.accuracy_score(y_true, y_pred)

0.33769776322967815

In [66]:
print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.78      0.88        18
           1       0.38      0.08      0.13        37
           2       0.28      0.65      0.39       231
           3       0.11      0.05      0.07        39
           4       0.71      0.60      0.65        80
           5       0.50      0.12      0.20         8
           6       0.31      0.30      0.31       128
           7       0.43      0.26      0.32        69
           8       0.15      0.17      0.16       150
           9       0.11      0.10      0.10        90
          10       0.67      0.43      0.53        46
          11       0.24      0.27      0.25        60
          12       0.34      0.20      0.25        51
          13       0.24      0.20      0.22        46
          14       0.15      0.08      0.10        50
          15       0.26      0.59      0.36        61
          16       0.38      0.08      0.13        38
          17       0.05    

In [67]:
metrics.roc_auc_score(y_true, y_score, average='weighted', multi_class='ovr')

0.8488167629037844

,ID,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TEST_0000,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TEST_0001,WT,WT,WT,WT,WT,R587Q,WT,WT,WT,...,WT,WT,WT,WT,WT,I383Sfs,WT,WT,WT,WT
2,TEST_0002,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TEST_0003,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TEST_0004,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,TEST_2541,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2542,TEST_2542,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2543,TEST_2543,WT,WT,WT,WT,WT,L217I,P221P P251P,R5M,G606D,...,S2049Vfs S1909Vfs,L232R,WT,WT,L305L,WT,N252I N251I,G679V,WT,WT
2544,TEST_2544,WT,WT,WT,WT,WT,WT,WT,WT,R1517H,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [78]:
#df_test['ID'] = test_ids
clf.predict(df_test)

array([21, 25,  8, ...,  2, 10,  6])

In [79]:
res = clf.predict(df_test)
res = encoder2.inverse_transform(res)
df_test1 = pd.read_csv('../input/test.csv')
res_df = pd.DataFrame ({'ID' : df_test1['ID'], 'SUBCLASS' : res})
res_df


,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,UCEC
2,TEST_0002,KIPAN
3,TEST_0003,LGG
4,TEST_0004,STES
...,...,...
2541,TEST_2541,COAD
2542,TEST_2542,HNSC
2543,TEST_2543,BRCA
2544,TEST_2544,LAML


In [80]:
res_df.SUBCLASS.value_counts()

SUBCLASS
BRCA      603
COAD      376
STES      237
KIPAN     236
OV        148
GBMLGG    133
KIRC       84
SKCM       81
UCEC       80
THCA       70
LGG        55
HNSC       52
LAML       52
LIHC       48
LUAD       44
PRAD       38
TGCT       34
PCPG       31
LUSC       28
CESC       28
BLCA       26
SARC       24
ACC        17
PAAD       16
THYM        3
DLBC        2
Name: count, dtype: int64

In [82]:
res_df.to_csv('../res-threshold/res1-0.csv',index = False)

In [26]:
res_df0['SUBCLASS'].value_counts()

SUBCLASS
BRCA      579
COAD      445
KIPAN     218
STES      187
GBMLGG    129
OV        100
SKCM       85
THCA       82
KIRC       82
HNSC       77
UCEC       74
LIHC       66
LGG        61
TGCT       55
PCPG       42
CESC       41
PRAD       38
LAML       34
LUAD       32
SARC       30
PAAD       28
LUSC       23
BLCA       21
ACC        11
THYM        5
DLBC        1
Name: count, dtype: int64

In [239]:
res_df = res_df0.copy()

## 01 Update Result with Zeros []

In [271]:
#zerosl = list(df_test[(df_test_2 == 0).all(axis=1)].index)
zerosl = [17, 180, 187, 214, 219, 294, 374, 446, 450, 545, 676, 696, 787, 855, 979, 1055, 1249, 1542, 1770, 1785, 1832, 1866, 1965, 2046, 2059, 2118, 2132, 2153, 2340, 2412, 2481]


[17,
 180,
 187,
 214,
 219,
 294,
 374,
 446,
 450,
 545,
 676,
 696,
 787,
 855,
 979,
 1055,
 1249,
 1542,
 1770,
 1785,
 1832,
 1866,
 1965,
 2046,
 2059,
 2118,
 2132,
 2153,
 2340,
 2412,
 2481]

In [272]:
res_df.loc[zerosl, 'SUBCLASS'] = 'THYM'

In [273]:
res_df

,ID,SUBCLASS
0,TEST_0000,STES
1,TEST_0001,UCEC
2,TEST_0002,BRCA
3,TEST_0003,LGG
4,TEST_0004,STES
...,...,...
2541,TEST_2541,COAD
2542,TEST_2542,PAAD
2543,TEST_2543,CESC
2544,TEST_2544,LAML


In [277]:
res_df.SUBCLASS.value_counts()

SUBCLASS
BRCA      625
COAD      532
KIPAN     221
STES      156
OV        139
GBMLGG    117
UCEC       83
THCA       71
LGG        70
HNSC       69
SKCM       67
KIRC       63
TGCT       40
CESC       37
THYM       36
LIHC       35
LAML       30
PRAD       29
BLCA       21
LUSC       20
PAAD       19
LUAD       18
ACC        17
PCPG       14
SARC       13
DLBC        4
Name: count, dtype: int64

In [275]:
res_df.loc[180]

ID          TEST_0180
SUBCLASS         THYM
Name: 180, dtype: object

## SAVE res_df with zero step

In [276]:
res_df.to_csv('../s20241011-2-step-1.csv',index = False)

In [35]:
############## <5 Finding SubSet

In [278]:
df = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

data = df.drop(columns=['ID', 'SUBCLASS'])
df_test.drop(columns='ID',inplace=True)

total = pd.concat([data, df_test])

f = lambda x : 0 if x == 'WT' else 1
total = total.map(f)

ns = df.shape[0]
data = total[:ns]
df_test = total[ns:]


In [243]:
df_test

,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2541,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2543,0,0,0,0,0,1,1,1,1,1,...,1,1,0,0,1,0,1,1,0,0
2544,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [279]:
#하나라도 1인  

# 딱 1개만 1인 행 필터링
one_1_rows = df_test[df_test.sum(axis=1) == 1]

# 각 행에서 1이 위치한 열의 이름과 인덱스를 구하는 코드
result = [(index, row.idxmax()) for index, row in one_1_rows.iterrows()]
result

[(8, 'SOX9'),
 (90, 'BRAF'),
 (110, 'LIG1'),
 (131, 'IDH2'),
 (170, 'PLK4'),
 (192, 'ZNF365'),
 (260, 'BRAF'),
 (352, 'PRKDC'),
 (366, 'VHL'),
 (390, 'DOCK2'),
 (435, 'CDCP1'),
 (619, 'GFPT2'),
 (655, 'CTNNA1'),
 (672, 'CD46'),
 (693, 'HRAS'),
 (701, 'HRAS'),
 (798, 'CCNL1'),
 (880, 'GNAO1'),
 (898, 'ATXN1'),
 (1023, 'MYH1'),
 (1034, 'CDKN2A'),
 (1071, 'VHL'),
 (1078, 'CACNA1A'),
 (1118, 'ACE'),
 (1202, 'VHL'),
 (1263, 'BRAF'),
 (1287, 'TG'),
 (1408, 'VHL'),
 (1426, 'VHL'),
 (1437, 'SREBF2'),
 (1560, 'CACNA1A'),
 (1690, 'LRPPRC'),
 (1761, 'VHL'),
 (1804, 'RAD21'),
 (1830, 'LIG1'),
 (1833, 'VHL'),
 (1851, 'NPM1'),
 (1879, 'VHL'),
 (1884, 'KMT2D'),
 (1979, 'NPM1'),
 (2056, 'VHL'),
 (2086, 'VHL'),
 (2114, 'BRCA2'),
 (2150, 'NPM1'),
 (2218, 'TP53'),
 (2383, 'NPM1'),
 (2447, 'CDH1'),
 (2512, 'SUCLG1')]

In [280]:
one_VHL_l=[]
one_BRAF_l = []
one_NPM1_l = []
one_LIG1_l = []
one_s = []
res_df1 = res_df.copy()

for i, c in result:
    print(i, c)
    if c =='VHL': 
        one_VHL_l.append(i)
        # 해당 컬럼에서 SUBCLASS값들의 분포를 구함
        q = ['VHL']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)] 
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS #KIRC
        res_df1.loc[one_VHL_l,'SUBCLASS'] = rs
    elif c=='BRAF': 
        one_BRAF_l.append(i)
        q = ['BRAF']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS #maybe 1
        res_df1.loc[one_BRAF_l,'SUBCLASS'] = rs
    elif c=='NPM1' : 
        one_NPM1_l.append(i)
        q = ['NPM1']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS#0 not 1
        res_df1.loc[one_NPM1_l,'SUBCLASS'] = rs
    elif c=='LIG1' : 
        one_LIG1_l.append(i)
        q = ['LIG1']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS#0 not 1
        res_df1.loc[one_LIG1_l,'SUBCLASS'] = rs
    elif c=='HRAS': 
        one_s.append(i)
        q = ['HRAS']
        rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS#0 not 1
        res_df1.loc[one_s,'SUBCLASS'] = rs
    #else:
        ###one_s.append(i)
        #row_no = i
        #q = [c]
        #rows_with_all_ones = df[data[q].eq(1).all(axis=1)]
        #rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS
        #res_df1.loc[i]['SUBCLASS'] = rs
    

res_df1.to_csv('../res-2000/res0-step2-ones.csv',index = False)
    

8 SOX9
90 BRAF
110 LIG1
131 IDH2
170 PLK4
192 ZNF365
260 BRAF
352 PRKDC
366 VHL
390 DOCK2
435 CDCP1
619 GFPT2
655 CTNNA1
672 CD46
693 HRAS
701 HRAS
798 CCNL1
880 GNAO1
898 ATXN1
1023 MYH1
1034 CDKN2A
1071 VHL
1078 CACNA1A
1118 ACE
1202 VHL
1263 BRAF
1287 TG
1408 VHL
1426 VHL
1437 SREBF2
1560 CACNA1A
1690 LRPPRC
1761 VHL
1804 RAD21
1830 LIG1
1833 VHL
1851 NPM1
1879 VHL
1884 KMT2D
1979 NPM1
2056 VHL
2086 VHL
2114 BRCA2
2150 NPM1
2218 TP53
2383 NPM1
2447 CDH1
2512 SUCLG1


In [ ]:
############# optional code

In [39]:
## TEST 데이터에서 1개의 컬럼만 1인 경우 찾기
one_1_rows = df_test[df_test.sum(axis=1) == 1]
# 각 열(column)별로 1이 있는 인덱스 리스트 생성
result_dict = {}
for column in one_1_rows.columns:
    indices = one_1_rows[one_1_rows[column] == 1].index.tolist()
    if indices:  # 인덱스가 비어있지 않으면 추가
        result_dict[column] = indices

# 딕셔너리를 DataFrame으로 변환
result_df = pd.DataFrame(list(result_dict.items()), columns=['gene', 'index_list'])
result_df


,gene,index_list
0,ACE,[1118]
1,ATXN1,[898]
2,BRAF,"[90, 260, 1263]"
3,BRCA2,[2114]
4,CACNA1A,"[1078, 1560]"
5,CCNL1,[798]
6,CD46,[672]
7,CDCP1,[435]
8,CDH1,[2447]
9,CDKN2A,[1034]


In [ ]:
q = 'VHL'
rows_with_all_ones = df[(data[q] == 1) & (data.drop(columns=q) == 0).all(axis=1)]
rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
#res_df1.loc[one_VHL_l,'SUBCLASS'] = rs
print(rs)

In [ ]:
## df_test 데이터에서 2개의 컬럼만 1인 경우 찾기
one_2_rows = df_test[df_test.sum(axis=1) == 2]
# 각 행에서 1이 위치한 열의 이름을 나열
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 리스트를 DataFrame으로 변환
result_df = pd.DataFrame(result, columns=['index', 'genes'])
result_df



In [ ]:
## df_test 데이터에서 2개의 컬럼만 1인 경우 찾기
one_2_rows = df_test[df_test.sum(axis=1) == 2]
# 각 행에서 1이 위치한 열의 이름을 나열
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 리스트를 DataFrame으로 변환
result_df = pd.DataFrame(result, columns=['index', 'genes'])
result_df



In [ ]:
## 1이 2개 이상 있는 행 필터링
one_2_rows = df_test[df_test.sum(axis=1) == 2]

# 각 행에서 1이 위치한 열의 이름을 모두 나열하는 코드
result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]

# 출력
print(result)


## Update Res_df with 2 / 3 / 4 / 5

In [282]:
## TEST DATA에서 1이 n개 있는 행 필터링
def find_test_n_ones(n):
    one_2_rows = df_test[df_test.sum(axis=1) == n]
    # 각 행에서 1이 위치한 열의 이름을 모두 나열하는 코드
    result = [(index, row[row == 1].index.tolist()) for index, row in one_2_rows.iterrows()]
    print(len(result))
    return result
print(find_test_n_ones(2))

47
[(74, ['CDH8', 'RYR2']), (75, ['HKDC1', 'UGCG']), (94, ['TGFBRAP1', 'TMPRSS6']), (100, ['DPYD', 'SYMPK']), (139, ['AADAT', 'JUP']), (384, ['DDX10', 'ELOVL5']), (387, ['CUL4A', 'ZFPM2']), (409, ['ARFGEF2', 'BRAF']), (534, ['CCR4', 'TLR7']), (565, ['BMP1', 'BRAF']), (637, ['CHEK2', 'UBC']), (733, ['BRAF', 'MXRA5']), (757, ['DCC', 'LRP1']), (772, ['CDON', 'MAP4K1']), (780, ['BRAF', 'LAMA4']), (796, ['NPM1', 'TTC39A']), (869, ['CCL25', 'HELZ2']), (878, ['PYGM', 'TGFBRAP1']), (899, ['ENPP1', 'SPRED2']), (911, ['IL1A', 'VWF']), (1042, ['ADAM23', 'RYR2']), (1065, ['MXD4', 'TCHH']), (1093, ['CPA2', 'DTX2']), (1267, ['CBR1', 'SLC19A1']), (1388, ['BRAF', 'LSP1']), (1390, ['EGLN2', 'NPM1']), (1520, ['ITIH1', 'PLG']), (1532, ['CDR2', 'XIAP']), (1555, ['HLA-DRB1', 'PREX1']), (1567, ['CTNS', 'IL12RB1']), (1571, ['DIO2', 'NPM1']), (1622, ['PROP1', 'ZNFX1']), (1634, ['SLIT1', 'ST6GAL1']), (1639, ['IRS4', 'LCP2']), (1756, ['COL12A1', 'TRIP10']), (1779, ['TGFBRAP1', 'TMPRSS6']), (1808, ['FGA', 'NIPBL

In [107]:
df.iloc[5618].apply(lambda x : x!='WT').agg(sum)

3

In [112]:
df.iloc[1954].SUBCLASS

'GBMLGG'

In [286]:
# exact matching은 보정
res_df2 = res_df1.copy()
for ns in range (1, 5):
    result_n = find_test_n_ones(ns)

    for i, qSet in result_n:
        # qSet에 해당하는 컬럼들만 값이 1인 행을 반환
        rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
        rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
        
        if rs.shape[0] == 1:
            rs = rs.loc[0].SUBCLASS
            if rs != res_df.loc[i]['SUBCLASS']:
                print(ns, rs, res_df1.loc[i]['SUBCLASS'], qSet)
                res_df2.loc[i,'SUBCLASS'] = rs
res_df2.to_csv('../res-2000/res0-step3-exactMatching.csv',index = False)


48
1 THCA BRCA ['PRKDC']
1 LAML BRCA ['RAD21']
1 KIRC BRCA ['KMT2D']
1 KIPAN BRCA ['SUCLG1']
47
2 LGG BRCA ['CTNS', 'IL12RB1']
60
3 KIRC UCEC ['MMP2', 'PTEN', 'TRIM58']
3 KIPAN PCPG ['CHEK2', 'PIK3CG', 'TAF1C']
3 GBMLGG LGG ['ATRX', 'GM2A', 'IDH1']
65
4 GBMLGG LAML ['CMKLR1', 'DVL2', 'IDH2', 'SCN10A']
4 GBMLGG LGG ['IDH1', 'LATS2', 'MYH8', 'PDGFA']
4 LGG BRCA ['ABCA4', 'ATRX', 'EXOC4', 'PTPN11']
4 LGG BRCA ['COL1A2', 'EDN2', 'KALRN', 'MXRA5']


In [117]:
res_df2.SUBCLASS.value_counts()

SUBCLASS
BRCA      552
COAD      376
KIPAN     239
STES      237
OV        147
GBMLGG    131
KIRC      110
SKCM       86
UCEC       78
THCA       68
LGG        62
HNSC       52
LAML       52
LIHC       47
LUAD       44
PRAD       34
THYM       33
TGCT       32
PCPG       28
CESC       28
LUSC       28
BLCA       26
SARC       21
ACC        17
PAAD       16
DLBC        2
Name: count, dtype: int64

In [ ]:
print(find_test_n_ones(4))

In [ ]:
print(find_test_n_ones(5))


In [ ]:
df_test

In [ ]:
# res 값 수정, 
# 문서 2,3,4,5
res_df2 = res_df1.copy()
result_n = find_test_n_ones(2)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        rs = rs.loc[0].SUBCLASS
        print(rs)
        res_df2.loc[i,'SUBCLASS'] = rs
    else: 
        #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
        rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
        if rows_with_all_ones.shape[0] > 1:
            try:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count')#.loc[].SUBCLASS## 0->1
                print(rs)
                #res_df2.loc[i,'SUBCLASS'] = rs
            except KeyError or ValueError:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS## 0->1
                res_df2.loc[i,'SUBCLASS'] = rs
                print(rs)
            
#res_df2.to_csv('res2-step3-two.csv',index = False)

In [ ]:
df

In [ ]:
result_n = find_test_n_ones(1)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        #rs = rs.loc[0].SUBCLASS
        print(rs)
        #res_df2.loc[i,'SUBCLASS'] = rs
    #else: 
    #    #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
    #    rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
    #    if rows_with_all_ones.shape[0] > 0 : 
    #        rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS
    #        print(rs)
    #        res_df2.loc[i,'SUBCLASS'] = rs


In [55]:
res_df2.to_csv('res-step4-three.csv',index = False)

In [48]:
res_df2.to_csv('res-step5-four.csv',index = False)

In [ ]:
result_n = find_test_n_ones(5)

for i, qSet in result_n:
    print(i, qSet)
    #option2) qSet에 해당하는 컬럼들만 값이 1인 행을 반환
    rows_with_only_qSet_ones = df[(data[qSet].eq(1).all(axis=1)) & (data.drop(columns=qSet).eq(0).all(axis=1))]
    rs = rows_with_only_qSet_ones['SUBCLASS'].value_counts().reset_index(name='count') #.loc[1].SUBCLASS #KIRC
    
    if rs.shape[0] > 0:
        rs = rs.loc[0].SUBCLASS
        print(rs)
        res_df2.loc[i,'SUBCLASS'] = rs
    else: 
        #option1) qSet에 해당하는 컬럼들이 포함된, 값이 1인 행을 반환
        rows_with_all_ones = df[data[qSet].eq(1).all(axis=1)]
        if rows_with_all_ones.shape[0] > 1 : 
            try: 
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[1].SUBCLASS
                print(rs)
                res_df2.loc[i,'SUBCLASS'] = rs
            except KeyError or ValueError:
                rs = rows_with_all_ones['SUBCLASS'].value_counts().reset_index(name='count').loc[0].SUBCLASS
                print(rs)
                res_df2.loc[i,'SUBCLASS'] = rs


In [58]:
res_df2.to_csv('res-step6-five.csv',index = False)